# Classificando o dataset MNIST usando MLPs e CNNs

### Membros

* Gabriel Pessoa
* Ícaro Guerra
* Lucas Barros
* Matheus Pessoa
* Rafael Mota

## Introdução
Esse relatório detalha o processo experimental para o desenvolvimento de uma solução para o problema de classificação de dígitos escritos manualmente do dataset MNIST usando Redes Neurais dos tipos: Multilayer Perceptron (MLP) e Convolutional Neural Network (CNN). O dataset MNIST consiste em 70 mil imagens 28x28 dos dígitos de 0 a 9, sendo 60 mil samples de treinamento e 10 mil samples de teste.

## Bibliotecas Utilizadas
Para a implementação utilzaremos as seguintes bibliotecas:


In [2]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, Adam

2021-07-11 14:05:08.819268: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-11 14:05:08.819296: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


* O Numpy e o Pandas são usados para a representação dos dados e para a implementação de funções auxiliares.

* Tensorflow e Keras são usados para a implementação e treinamento das Redes Neurais.

## Carregamento dos Dados
O código a seguir carrega os dados do dataset, adapta ao formato desejado e define os parâmetros globais sobre os dados.

In [3]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Parâmetros de Treinamento
Aqui definimos os parâmetro de treinamento que serão utilizados para todas as redes neurais, a escolha dessa quantidade de epochs e batch_size, foi feita para que tenhamos maior perfomance e mais dinamicidade no experimento com os treinamentos.

In [4]:
# Training parameters
epochs = 15
batch_size = 128
verbose = 0

metrics = ["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]

2021-07-11 14:05:13.500922: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-11 14:05:13.500957: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-11 14:05:13.501023: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LARANJAL-016153): /proc/driver/nvidia/version does not exist
2021-07-11 14:05:13.501358: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Funções de Avaliação
Aqui definimos a duas funções que vamos usar para avaliar as soluções, a primeira função `accuracy_per_class` retorna a métrica de *acurácia* para cada uma das classes de resposta, que no nosso caso são os possíveis dígitos de 0 a 9. A segunda função `accuracy_precision_recall` retorna as métricas *acurácia*, *precision* e *recall* total.

Um breve descrição das métricas citadas seria:

* *acurácia*: A proporção de predição corretas com o total de casos.
* *precision*: A proporção de predições positivas corretas com o total de predições positivas da classe.
* *recall*: A proporção de predições positivas corretas com o total de casos da classe.

In [5]:
def accuracy_per_class(model):
    y_pred = model.predict(x_test)

    correct = [0] * num_classes
    total = [0] * num_classes

    for real, pred in zip(y_test.argmax(axis=1), y_pred.argmax(axis=1)):
        if real == pred:
            correct[real] += 1
        total[real] += 1

    accuracies = []
    for correct, total in zip(correct, total):
        accuracies.append(correct / total)

    return pd.Series(data=accuracies, index=range(0,num_classes), name="Accuracy per class")

def accuracy_precision_recall(model):
    _, acc, prec, rec = model.evaluate(x_test, y_test,verbose=0)

    return pd.Series(data=[acc, prec, rec],index=["Accuracy", "Precision", "Recall"])

## Experimentação

A seguir começaremos a fase de experimentação, testando diferentes modelos de Redes Neurais e de aprendizado, para achar uma solução para o problema de classificação do dataset MNIST.


### MLP 1
Primeiramente, vamos usar uma rede neural MLP simples, com uma camada de entrada com um neuron para cada um dos pixels da imagem, uma camada oculta com 50 neurons, e uma camada de saída com ativação softmax, com um neuron para cada possível saída, no nosso caso 10. Essa estrutura vai se repetir em futuras redes, podendo variar os parâmetros de aprendizado e os parâmetros e a estrutura das camadas ocultas.

In [35]:
model = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(50),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=metrics)

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 50)                39250     
_________________________________________________________________
dense_12 (Dense)             (None, 10)                510       
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________


In [36]:

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)


In [37]:
accuracy_per_class(model)

0    0.981633
1    0.978855
2    0.890504
3    0.907921
4    0.929735
5    0.867713
6    0.948852
7    0.922179
8    0.877823
9    0.894945
Name: Accuracy per class, dtype: float64

Com o primeiro modelo obtemos o seguinte resultado, um rede neural simples já acerta grande parte dos casos do Dataset, mas podemos melhorar isso.

In [38]:
accuracy_precision_recall(model)

Accuracy     0.921100
Precision    0.938185
Recall       0.907600
dtype: float64

### MLP 2
A seguir verificamos o impacto de aumentar a quantidade de neurons na mesma Layer, conservando os demais parâmetros.

In [39]:
model2 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(400),
    layers.Dense(num_classes, activation="softmax")
])

model2.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=metrics)

model2.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 400)               314000    
_________________________________________________________________
dense_14 (Dense)             (None, 10)                4010      
Total params: 318,010
Trainable params: 318,010
Non-trainable params: 0
_________________________________________________________________


In [40]:
model2.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

Nesse caso podemos verificar que houve um pequeno ganho de desempenho.

In [42]:
accuracy_precision_recall(model2)

Accuracy     0.92260
Precision    0.93994
Recall       0.90770
dtype: float64

### MLP 3
Podemos tentar também adicionar mais uma camada oculta de Neurons:

In [43]:
model2 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(300),
    layers.Dense(100),
    layers.Dense(num_classes, activation="softmax")
])

model2.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=metrics)

model2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_16 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_17 (Dense)             (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [44]:
model2.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

Verificamos que houve um pequeno ganho de perfomance, e que o processo de treinamento e predição ficaram bem mais lentos.

In [45]:
accuracy_precision_recall(model2)

Accuracy     0.924200
Precision    0.938123
Recall       0.912700
dtype: float64

### MLP 4
Outro parâmetro que podemos variar é a função de ativação, o default para a biblioteca é a função linear, que não é muito recomendada para problemas, não linearmente separáveis. Vamos verificar com o a função sigmoid

In [47]:
model3 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(400, activation="sigmoid"),
    layers.Dense(num_classes, activation="softmax")
])

model3.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=metrics)

model3.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 400)               314000    
_________________________________________________________________
dense_19 (Dense)             (None, 10)                4010      
Total params: 318,010
Trainable params: 318,010
Non-trainable params: 0
_________________________________________________________________


In [48]:
model3.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

Usando a função de ativação sigmoid houve uma piora no desempenho.

In [49]:
accuracy_precision_recall(model3)

Accuracy     0.914100
Precision    0.939669
Recall       0.890900
dtype: float64

### MLP 5
Vamos testar agora com a função de ativação ReLU:

In [50]:
model4 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(400, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model4.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=metrics)

model4.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 400)               314000    
_________________________________________________________________
dense_21 (Dense)             (None, 10)                4010      
Total params: 318,010
Trainable params: 318,010
Non-trainable params: 0
_________________________________________________________________


In [51]:
model4.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

Com a função de ativação ReLU houve uma melhora considerável em relação às execução usando outras funções de ativação.

In [52]:
accuracy_precision_recall(model4)

Accuracy     0.958300
Precision    0.968482
Recall       0.949500
dtype: float64

### MLP 6
Agora, podemos tentar adicionar uma nova camada na rede, dessa vez usando a função de ativação ReLU:


In [53]:
model5 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(400, activation="relu"),
    layers.Dense(300, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model5.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=metrics)

model5.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_10 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 400)               314000    
_________________________________________________________________
dense_23 (Dense)             (None, 300)               120300    
_________________________________________________________________
dense_24 (Dense)             (None, 10)                3010      
Total params: 437,310
Trainable params: 437,310
Non-trainable params: 0
_________________________________________________________________


In [54]:
model5.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

O tempo para treinamento e predição aumentou mas houve um melhora considerável na acurácia da rede usando 2 camadas de neurons com a função de ativação ReLU.

In [55]:
accuracy_precision_recall(model5)

Accuracy     0.971000
Precision    0.975856
Recall       0.966000
dtype: float64

### MLP 7

Em seguida, podemos testar variar a taxa de aprendizado, o valor padrão para esse parâmetro na biblioteca Keras é 0.01, vamos experimentar aumentar esse valor:

In [22]:
model6 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(400, activation="relu"),
    layers.Dense(300, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model6.compile(loss="categorical_crossentropy", optimizer=SGD(0.2), metrics=metrics)

model6.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 400)               314000    
_________________________________________________________________
dense_13 (Dense)             (None, 300)               120300    
_________________________________________________________________
dense_14 (Dense)             (None, 10)                3010      
Total params: 437,310
Trainable params: 437,310
Non-trainable params: 0
_________________________________________________________________


In [28]:
model6.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

Obtemos uma melhora em relação ao resultado anterior, isso provavelmente acontece por estarmos executando com uma quantidade limitada de épocas, fazendo com que uma taxa de ativação maior faça a rede neural convergir mais rápidamente.

In [29]:
accuracy_precision_recall(model6)

Accuracy     0.983100
Precision    0.983684
Recall       0.982700
dtype: float64

### MLP 8

Podemos também usar uma técnica de regularização chamada de Dropout, que no processo de treinamento "ignora" uma porcentagem dos neurons a cada iteração. Essa técnica tem como objetivo distribuir o "trabalho" de reconhecimento das classes entre diversos Neurons, o que pode ajudar a reduzir o overfitting. 

In [7]:
model7 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(400, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(300, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model7.compile(loss="categorical_crossentropy", optimizer=SGD(0.2), metrics=metrics)

model7.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 400)               314000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 300)               120300    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                3010      
Total params: 437,310
Trainable params: 437,310
Non-trainable params: 0
__________________________________________________

In [8]:
model7.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

2021-07-11 14:05:44.010606: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-11 14:05:44.012315: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2299965000 Hz


Obtemos uma melhora no desempenho ao usar essa técnica.

In [9]:
accuracy_precision_recall(model7)

Accuracy     0.986100
Precision    0.987174
Recall       0.985200
dtype: float64

### MLP 9

Podemos também tentar mudar o algoritmo de treinamento, um possível algoritmo é o Adam:

In [10]:
model8 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(400, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(300, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model8.compile(loss="categorical_crossentropy", optimizer=Adam(0.2), metrics=metrics)

model8.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 400)               314000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 300)               120300    
_________________________________________________________________
dense_6 (Dense)              (None, 10)                3010      
Total params: 437,310
Trainable params: 437,310
Non-trainable params: 0
________________________________________________

In [11]:
model8.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

Analisando os resultados, podemos deduzir que o algoritmo Adam não funciona com learning rates altos para esse conjunto de dados.

In [12]:
accuracy_precision_recall(model8)

Accuracy     0.0982
Precision    0.0000
Recall       0.0000
dtype: float64

### MLP 10

Tentamos em seguida executar o com o algoritmo de aprendizado Adam com a learning rate default.

In [17]:
model9 = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(400, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(300, activation="relu"),
    layers.Dense(num_classes, activation="softmax")
])

model9.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=metrics)

model9.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 784)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 400)               314000    
_________________________________________________________________
dropout_8 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 300)               120300    
_________________________________________________________________
dense_12 (Dense)             (None, 10)                3010      
Total params: 437,310
Trainable params: 437,310
Non-trainable params: 0
________________________________________________

In [18]:
model9.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

Com o algoritmo de treinamento Adam, tivemos praticamente a mesmas acurácia, portanto não teve uma melhora significativa. 

In [29]:
accuracy_precision_recall(model9)

Accuracy     0.986400
Precision    0.986793
Recall       0.986300
dtype: float64

### CNN 1

Agora vamos tentar usar uma Rede Neural Convolucional, que tem uma vantagem em relação às MLPS, pois ela considera as estrutura da matriz de entrada, primeiramente, vamos usar uma CNN simples com apenas uma camada convolucional e uma camada de pooling.

In [31]:
model10 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model10.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=metrics)

model10.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 10)                54090     
Total params: 54,410
Trainable params: 54,410
Non-trainable params: 0
_________________________________________________________________


In [33]:
model10.fit(x_train, y_train, batch_size=batch_size, epochs=10, validation_split=0.1, verbose=verbose)

Tivemos uma boa acurácia de inicio, mas ainda ainda mais baixa do que nosso MLP inicial.

In [34]:
accuracy_precision_recall(model10)

Accuracy     0.981600
Precision    0.983049
Recall       0.980100
dtype: float64

### CNN 2

Podemos também, aumentar a janela de convolução:

In [50]:
model14 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model14.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=metrics)

model14.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 10816)             0         
_________________________________________________________________
dense_22 (Dense)             (None, 10)                108170    
Total params: 108,810
Trainable params: 108,810
Non-trainable params: 0
_________________________________________________________________


In [51]:
model14.fit(x_train, y_train, batch_size=batch_size, epochs=10, validation_split=0.1, verbose=verbose)

Teve uma pequena melhor na acurácia ao aumentar o tamanho da janela:

In [53]:
accuracy_precision_recall(model14)

Accuracy     0.983200
Precision    0.984665
Recall       0.982400
dtype: float64

### CNN 3

Podemos também variar o parâmetro de stride, que determina o quantos valores serão "pulados" para gerar as janelas de convolução.

In [41]:
model13 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), stride=(2, 2), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model13.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=metrics)

model13.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                54090     
Total params: 54,410
Trainable params: 54,410
Non-trainable params: 0
_________________________________________________________________


In [43]:
model13.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

Tivemos uma melhora não significativa.

In [44]:
accuracy_precision_recall(model13)

Accuracy     0.983400
Precision    0.984858
Recall       0.982100
dtype: float64

### CNN 4

Podemos também tentar adicionar mais uma camada de convolução:

In [35]:
model11 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model11.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=metrics)

model11.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                16010     
Total params: 34,826
Trainable params: 34,826
Non-trainable params: 0
__________________________________________________

In [36]:
model11.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

Adicionando uma camada de convolução obtivemos uma melhora considerável.

In [37]:
accuracy_precision_recall(model11)

Accuracy     0.989200
Precision    0.990382
Recall       0.988500
dtype: float64

### CNN 5

Podemos também tentar adicionar uma camada de Dropout para os Neurons de Saída.

In [45]:
model12 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model12.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=metrics)

model12.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 1600)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)              

In [46]:
model12.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=verbose)

Adicionando a camada de Dropout conseguimos a nossa melhor acurácia até agora.

In [48]:
accuracy_per_class(model12)

0    0.994898
1    0.995595
2    0.989341
3    0.996040
4    0.993890
5    0.989910
6    0.983299
7    0.990272
8    0.988706
9    0.983152
Name: Accuracy per class, dtype: float64

In [47]:
accuracy_precision_recall(model12)

Accuracy     0.990600
Precision    0.991884
Recall       0.989900
dtype: float64